In [ ]:
"""
Evaluate model on test set and generate Evidently HTML report
"""
import os
import joblib
import pandas as pd
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, ClassificationPreset
from evidently.pipeline.column_mapping import ColumnMapping

def run(test_csv="data/processed/test.csv", model_path="models/model.joblib", out_report="reports/evidently_report.html"):
    os.makedirs(os.path.dirname(out_report), exist_ok=True)
    model = joblib.load(model_path)
    df_test = pd.read_csv(test_csv)

    # Create reference / production split from test
    reference = df_test.sample(frac=0.5, random_state=42).reset_index(drop=True)
    production = df_test.drop(reference.index).reset_index(drop=True)

    # Add predictions
    X_ref = reference.drop(columns=["PlacementStatus"])
    X_prod = production.drop(columns=["PlacementStatus"])

    reference = reference.copy()
    production = production.copy()
    reference["prediction"] = model.predict(X_ref)
    production["prediction"] = model.predict(X_prod)

    mapping = ColumnMapping(target='PlacementStatus', prediction='prediction')

    report = Report(metrics=[DataDriftPreset(), ClassificationPreset()])
    report.run(reference_data=reference, production_data=production, column_mapping=mapping)
    report.save_html(out_report)
    print(f"Evidently report saved to {out_report}")

if __name__ == "__main__":
    run()
